# Setting

In [ ]:
import requests
import pandas as pd
import time
from tqdm import tqdm

In [339]:
api_key = ''
save_path = '/content/drive/MyDrive/Git/League_of_Legends_Match_Prediction/Data'

In [340]:
def get_league_summonerInfo(league = 'challenger'):
    """
    Get the Summoner ID Series from Riot api server.

    Parameters
    ----------
    league : string

    Returns
    ----------
    Series
    """
    league_list = ['challenger','grandmaster','master']
    if league not in league_list:
        raise ValueError("'{}' is not a valid value for name; supported values are {}".format(league, ', '.join(league_list)))

    url_ = 'https://kr.api.riotgames.com/lol/league/v4/'+league+'leagues/by-queue/RANKED_SOLO_5x5/' + '?api_key=' + api_key
    json_ = requests.get(url_).json()
    summonerId = pd.DataFrame(json_['entries']).loc[:,['summonerId','summonerName']]

    return summonerId

In [341]:
def get_puuid(summonerName):
    """
    Get the puuid Series from Riot api server.

    Parameters
    ----------
    summonerName : string

    Returns
    ----------
    string
    """
    url_ = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + summonerName + '?api_key=' + api_key
    request_ = requests.get(url_)

    while request_.status_code == 429:
        time.sleep(5)
        request_ = requests.get(url_)
    
    if request_.status_code == 200:
        json_ = request_.json()
        ret_ = json_['puuid']
    else:
        ret_ = None

    return ret_

In [ ]:
def get_timeline(matchId):
    """
    Get a match timeline from Riot api server.

    Parameters
    ----------
    matchId : string

    Returns
    ----------
    Dict
    """
    url_ = 'https://asia.api.riotgames.com/lol/match/v5/matches/' +matchId + '/timeline' + '?api_key=' + api_key
    request_ = requests.get(url_)
    json_ = request_.json()

    return json_